In [1]:
from random import randint
from collections import defaultdict
from uuid import uuid4
from dataclasses import dataclass

import matplotlib.pyplot as plt
from ipynb.fs.full.Observer import CaptureMachine, TrackedObject, CaptureConfiguration, Camera, hStackImages, vStackImages
import dma.MechaCombat as mc
QuantumSystem = mc.QuantumSystem

In [2]:
@dataclass
class QuantumObject(TrackedObject):
    name: str
    objectType: str

    def __eq__(self, other):
        return super().__eq__(other)
    
    def __post_init__(self):
        super().__post_init__()
        self.qid = str(uuid4())
        
    def update(self, trackedObjDef: TrackedObject):
        self.changeSet = trackedObjDef.changeSet

In [3]:
class HarmonyMachine(CaptureMachine):
    states = ["idle", "unstable", "illegal"]
    modes = ["passive", "calibrate", "add", "move", "action"]
    def __init__(self, captureConfiguration: CaptureConfiguration):
        super().__init__(captureConfiguration)
        self.hc = self.cc
        mc.qs.reset()
        self.newObjectBuffer = []
    
    def isLegal(self, classifiedChange):
        return True
    
    def storeNewObject(self, objDef):
        assert objDef.isNewObject, "Cannot add non-new objects"
        self.newObjectBuffer.append(objDef)
        self.lastMemory = objDef
        
        for camNum, change in objDef.changeSet.items():
            self.cc.cameras[camNum].commitChange(change)
        
        self.transitions.append({
            "obj": objDef,
            "cycle": self.cycleCounter,
            "cameraChanges":  {camNum: {"ref": cam.referenceFrame, "fin": cam.mostRecentFrame}
                               for camNum, cam in self.cc.cameras.items()}})
    
    def commitChanges(self, objDef):
        try:
            existingIndex = self.memory.index(objDef.previousVersion())
            existingObj = self.memory[existingIndex]
            print(f"Updating Memory {existingIndex}")
            existingObj.update(objDef)
        except ValueError:
            raise Exception("Cannot commit to unrecognized memory")
        self.lastMemory = existingObj
        
        for camNum, change in objDef.changeSet.items():
            self.cc.cameras[camNum].commitChange(change)
        
        self.transitions.append({
            "obj": objDef,
            "cycle": self.cycleCounter,
            "cameraChanges":  {camNum: {"ref": cam.referenceFrame, "fin": cam.mostRecentFrame}
                               for camNum, cam in self.cc.cameras.items()}})

    def annotateObject(self, objName, objType):
        caps = {cap.oid: cap for cap in cm.newObjectBuffer}
        cap = caps[oid]
        center = cc.rsc.trackedObjectToRealCenter(cap)
        qObj = QuantumObject(cap.changeSet, objName, objType)
        factory = ObjectFactories[objType]
        factory(objName)
        mc.Location.set_location(objName, json.dumps(list(center) + [0]))
        cm.memory.append(qObj)
        cm.newObjectBuffer.remove(cap)
    
    def cycle(self):
        try:
            print(f"Starting Cycle {self.cycleCounter:5} -- {self}")
            self.cycleCounter += 1
            nextState = "idle"
            self.hc.capture()
            if self.mode == "passive":
                self.hc.setReference()
                return
            # else: mode in ["calibrate", "add", "move", "attack"]

            changes = self.referenceFrameDeltas()
            classification = None
            if changes.empty:
                self.hc.setReference()
            else:
                nextState = "unstable"
                if changes == self.lastChanges:  # TODO: and cameraChangeOverlap
                    classification = self.classifyChanges(changes)
                    if self.mode in ["move", "action"]:
                        try:
                            assert not classification.isNewObject, f"Cannot add objects in {self.mode}"
                            assert self.isLegal(classification), f"{classificaton} is an illegal move"
                            nextState = "idle"
                            self.commitChanges(classification)
                            self.hc.setReference()
                        except AssertionError:
                            nextState = "unstable"
                    elif self.mode == "add":
                        nextState = "idle"
                        try:
                            self.storeNewObject(classification)
                            self.hc.setReference()
                        except Exception as e:
                            print(f"Add Object Failure: {e}")
                            nextState = "unstable"
                    elif self.mode == "calibrate":
                        try:
                            self.calibrateToObject(classification)
                        except AssertionError as ae:
                            print(f"Failed Calibration: {ae}")
                        nextState = "idle"
                        self.hc.setReference()
                    else:
                        raise Exception("Unrecognized State")
            self.state = nextState
            self.cycleCounter += 1
            self.lastChanges = changes
        except:
            from traceback import format_exc
            print("CYCLE FAILURE!!!")
            print(format_exc())
            raise
    
    def actionMode(self):
        self.mode = "action"
    
    def moveMode(self):
        self.mode = "move"
    
    def addMode(self):
        self.mode = "add"
    
    def passiveMode(self):
        self.mode = "passive"
    
    def startCalibration(self):
        self.calibrationPts = []
        self.mode = "calibrate"
    
    def abortCalibration(self):
        self.calibrationPts = []
        self.mode = "passive"
            
    def cycleForChange(self):
        self.mode = "track"
        startLen = len(self.transitions)
        while len(self.transitions) == startLen:
            self.cycle()
        self.mode = "idle"
        return self.lastMemory
            
    def __repr__(self):
        return f"CapMac -- {self.mode} {str(len(self.calibrationPts)) + ' ' if self.mode == 'calibrate' else ''}{self.state}"

In [4]:
cc = CaptureConfiguration()
cc.loadConfiguration()
cameras = cc.cameras
cm = HarmonyMachine(cc)